# 预处理数据

## 1.1  Merge all excel data

In [1]:
import pandas as pd
import os, sys
import glob
import pandas as pd
import os, sys
import glob

DIR='/workspace/match_questions/'
FILES = glob.glob(DIR+'*.xlsx') + glob.glob(DIR+'*.xls')
all_df = pd.DataFrame(columns=['组数', '轮数', '上文', 'index', 'owner', 'question_1', 'question_2',
       'question_3', 'question_4', 'question_5', 'question_6', 'question_7',
       'question_8', 'question_9', 'question_10'])
keys = {}
index = 0
for file in FILES:
    if file.find("all")  != -1:
        continue
    if file.find('12.xlsx') != -1:
        df = pd.read_excel(file, sheet_name="Sheet1")
    elif file.find('12.xls') == -1:
        df = pd.read_excel(file)
    columns_names = list(set(df.columns) - set(['组数', '轮数', '上文', 'index', 'owner']))
    for idx in range(df.shape[0]):
        key = "{}_{}".format(df.loc[idx, '组数'], df.loc[idx, '轮数'])
        if pd.isnull(df.loc[idx][columns_names[0]]):
            continue
        if key in keys:
            print(columns_names[0], df.loc[idx][columns_names[0]])
            print("conflict/duplicate row{}: {}/{} --> {}/{}".format(key, file, idx, keys[key][0], keys[key][1]))
        else:
            keys[key] = (file, idx)
        
        all_df.loc[index,('组数', '轮数', '上文', 'index', 'owner')] = df.loc[idx, ['组数', '轮数', '上文', 'index', 'owner']]
        for jdx, name in enumerate(columns_names):
            if jdx < 10 and not pd.isnull(df.loc[idx][name]):
                all_df.loc[index]['question_%d'%(jdx+1)] = df.loc[idx][name]   
        index += 1
all_df.to_excel(DIR+"/all.xlsx")

question_2 那还有哪些比较实在的赔偿呢？
conflict/duplicate row327_7: /workspace/match_questions/5.xls/472 --> /workspace/match_questions/5.xls/471
question_2 关于医疗的赔偿项目是怎么安排？
conflict/duplicate row414_10: /workspace/match_questions/11.xls/1220 --> /workspace/match_questions/12.xlsx/1220


## 1.2 Build vocab and convert text to id

In [4]:
all_df = pd.read_excel(DIR+"/all.xlsx", sheet_name="Sheet1")
vocab_output_file = 'work/vocab.txt'
output_file = 'work/doc_idx.txt'
all_df.head(n=5)

,组数,轮数,上文,index,owner,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9,question_10
0,309,7,那未成年人的保金是不是也要高一些？,4,黄文,孩子的保险费是否更高？,未成年要交更高的保金吗,未成年人的保险金是否更高？,保金对未成年人来说是否更高,那未成年的保金是不是就更高了？,那未成年人要交更高的保费吗,未成年人的保险金更高吗？,未成年人的保险费是否更高？,未成年人是不是要缴纳更高的保金,那未成年人的保金要高一些吗？
1,309,10,一般来说，一年一次吧。,4,黄文,通常每年一次。,总体而言，每年一次。,一般来说，每年一次。,一般来说，每年一次,一般每年一次。,一年一次吧。,一般来说，每年一次吧。,一般来说，一年一次。,通常来说，一年一次。,每年一次。
2,310,2,有哪几种类型来选择呢？,4,黄文,哪些类型可供选择？,什么样的类型可以选择？,有哪些类型可供选择？,我可以选择哪些类型？,什么类型可供选择？,什么是不同类型的选择？,你可以选择什么类型？,有哪些类型选择？,那里有什么类型？,我可以选择哪些类型？
3,310,4,如果因遗失信用卡而引致被盗损失我能获得多少赔付呢？,4,黄文,如果我丢了信用卡，卡被盗用了，我可以拿到多少赔偿？,如果我丢失了信用卡而导致信用卡盗用，会有多少赔付呢？,如果我因信用卡丢失被盗，可以得到多少赔付？,由于信用卡丢失而导致被盗，我的损失可以得到赔偿吗,如果我的卡被盗是因为信用卡丢失，我能得到多少赔付？,如果我的可被盗，是因为信用卡遗失，我可以获得赔付吗,由于信用卡丢失引发的损失，我可以拿到赔偿吗？,如果因遗失信用卡而引致被盗损失，我能获得多少赔付呢？,如果因为信用卡丢失被窃，造成的损失可以拿到多少赔偿呢,要是我信用卡丢了，然后被盗用，这些损失我能得到多少赔偿呢
4,310,5,那有什么要求吗？,4,黄文,有什么需求？,要求是什么呢,有什么要求？,那有哪些要求需要满足呢？,要求有哪些,我需要满足什么要求吗,有什么条件？,有什么条件吗,那有什么样的条件？,我需要满足哪些条件呢


In [5]:
all_df.shape

(1545, 15)

In [19]:
from dingding.tools import nlp_utils
from hanziconv import HanziConv
def clean_sentence(sentence):
    sentence = nlp_utils.full2half(sentence)
    sentence = HanziConv.toSimplified(sentence)
    return sentence

all_sentences = []
for idx in range(all_df.shape[0]):
    sentences = []
    s = all_df.loc[idx, u'上文']
    s1 = clean_sentence(s)
    sentences.append(s1)
    for jdx in range(1,11):
        s = all_df.loc[idx, 'question_%d'%(jdx)]
        if (not pd.isna(s)) and (not pd.isnull(s)):
            s1 = clean_sentence(s)
            sentences.append(s1)
    all_sentences.append(sentences)

In [20]:
len(all_sentences)

1545

In [34]:
# build vocab
import gensim
import jieba
all_seg_sentences1 = []
all_seg_sentences2 = []
for idx in range(len(all_sentences)):
    seg_sentences = []
    for s in all_sentences[idx]:
        seg = jieba.tokenize(s)
        seg_sentences.append([w[0] for w in seg])
    all_seg_sentences1.append(seg_sentences)
    all_seg_sentences2 += seg_sentences

In [53]:
from gensim.corpora import Dictionary as corpora_Dictionary
dct = corpora_Dictionary(all_seg_sentences2)
with open(vocab_output_file, 'w') as f:
    for idx in range(len(dct)):
        f.write("{}\\{}\n".format(idx, dct[idx]))

all_seg_idx = []

for idx in range(len(all_seg_sentences1)):
    idx_sentences = []
    for s in all_seg_sentences1[idx]:
        sentence = dct.doc2idx(s)
        idx_sentences.append(sentence)
    all_seg_idx.append(idx_sentences)

with open(output_file, 'w') as f:
    for idx in range(len(all_seg_idx)):
        for s in all_seg_idx[idx]:
            f.write("{}\n".format(" ".join([str(x) for x in s])))

In [60]:
max([max([len(j) for j in all_seg_idx[i]]) for i in range(len(all_seg_idx))])

38

In [59]:
max([j for j in all_seg_idx[0]])

[404, 774, 141, 124, 309, 6, 979, 26]

## 1.3 split data into train/test 
###  current method:  similar sentences [s0, s1, ..., s10] --> (s0, s1), (s1, s2), ..., (s10, s0)

In [55]:
train_test_split_ratio = 0.8
train_file = 
import random
random.shuffle(all_seg_idx)
